<a href="https://colab.research.google.com/github/JuliaCherep/innopolis/blob/Test1/%D0%94%D0%B713.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 40.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=e046c1e0cb7ffb318325cb6234b101f56901adecf4592833ee736c3484b82780
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pandas as pd
import numpy as np

from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local') 
spark = SparkSession(sc)

In [4]:
df = spark.read.format("com.databricks.spark.csv").options(sep=",", header=True, quote="").csv('iris.csv')
df.show()

+-------------+---------------+----------------+---------------+------------+
|"sepal.length|""sepal.width""|""petal.length""|""petal.width""|""variety"""|
+-------------+---------------+----------------+---------------+------------+
|         "5.1|            3.5|             1.4|             .2| ""Setosa"""|
|         "4.9|              3|             1.4|             .2| ""Setosa"""|
|         "4.7|            3.2|             1.3|             .2| ""Setosa"""|
|         "4.6|            3.1|             1.5|             .2| ""Setosa"""|
|           "5|            3.6|             1.4|             .2| ""Setosa"""|
|         "5.4|            3.9|             1.7|             .4| ""Setosa"""|
|         "4.6|            3.4|             1.4|             .3| ""Setosa"""|
|           "5|            3.4|             1.5|             .2| ""Setosa"""|
|         "4.4|            2.9|             1.4|             .2| ""Setosa"""|
|         "4.9|            3.1|             1.5|             .1|

In [5]:
nC= ["sepal_length","sepal_width","petal_length","petal_width","variety"]
df_n = df.toDF(*nC)
df_n.show()

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    variety|
+------------+-----------+------------+-----------+-----------+
|        "5.1|        3.5|         1.4|         .2|""Setosa"""|
|        "4.9|          3|         1.4|         .2|""Setosa"""|
|        "4.7|        3.2|         1.3|         .2|""Setosa"""|
|        "4.6|        3.1|         1.5|         .2|""Setosa"""|
|          "5|        3.6|         1.4|         .2|""Setosa"""|
|        "5.4|        3.9|         1.7|         .4|""Setosa"""|
|        "4.6|        3.4|         1.4|         .3|""Setosa"""|
|          "5|        3.4|         1.5|         .2|""Setosa"""|
|        "4.4|        2.9|         1.4|         .2|""Setosa"""|
|        "4.9|        3.1|         1.5|         .1|""Setosa"""|
|        "5.4|        3.7|         1.5|         .2|""Setosa"""|
|        "4.8|        3.4|         1.6|         .2|""Setosa"""|
|        "4.8|          3|         1.4| 

In [6]:
df_n = df_n.withColumn("sepal_length", regexp_replace(col("sepal_length"), '"', ""))\
    .withColumn("variety", regexp_replace(col("variety"), '"', ""))


df_n=df_n \
    .withColumn('sepal_length', col('sepal_length').cast(DoubleType())) \
    .withColumn('sepal_width', col('sepal_width').cast(DoubleType())) \
    .withColumn('petal_length', col('petal_length').cast(DoubleType())) \
    .withColumn('petal_width', col('petal_width').cast(DoubleType()))
df_n.printSchema()    
df_n.show(10)

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety: string (nullable = true)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
+

In [7]:
df_n.summary().show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|
|    25%|               5.1|                2.8|               1.6|               0.3|     null|
|    50%|               5.8|                3.0|               4.3|               1.3|     null|
|    75%|               6.4|                3.3|               5.1|               1.8|     null|
|    max|               7.9|  

In [8]:
pipeline = Pipeline(stages = 
[
  StringIndexer(inputCol='variety', outputCol='varietyInd'),
  VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol='Features')
]
)

In [9]:
pipelineTrained = pipeline.fit(df_n)

In [10]:
pipelineTrained.transform(df_n).show()

+------------+-----------+------------+-----------+-------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|varietyInd|         Features|
+------------+-----------+------------+-----------+-------+----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|       0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|       0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|       0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|       0.0|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| Setosa|       0.0|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| Setosa|       0.0|[5.0,3.4,1.5,0.2]|
|         4.4|       

In [13]:
df_f = pipelineTrained.transform(df_n)

In [14]:
train, test = df_f.randomSplit([0.8, 0.2], seed=12345)

In [15]:
train.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|
|         4.5|        2.3|         1.3|        0.3|    Setosa|       0.0|[4.5,2.3,1.3,0.3]|
|         4.6|        3.1|         1.5|        0.2|    Setosa|       0.0|[4.6,3.1,1.5,0.2]|
|         4.6|        3.4|         1.4|        0.3|    Setosa|       0.0|[4.6,3.4,1.4,0.3]|
|         4.6|        3.6|         1.0|        0.2|    Setosa|       0.0|[4.6,3.

In [16]:
test.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|
+------------+-----------+------------+-----------+----------+----------+-----------------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|
|         5.0|        3.0|         1.6|        0.2|    Setosa|       0.0|[5.0,3.0,1.6,0.2]|
|         5.0|        3.2|         1.2|        0.2|    Setosa|       0.0|[5.0,3.2,1.2,0.2]|
|         5.0|        3.5|         1.3|        0.3|    Setosa|       0.0|[5.0,3.5,1.3,0.3]|
|         5.1|        3.5|         1.4|        0.3|    Setosa|       0.0|[5.1,3.5,1.4,0.3]|
|         5.4|        3.4|         1.5|        0.4|    Setosa|       0.0|[5.4,3.4,1.5,0.4]|
|         5.4|        3.9|         1.3|        0.4|    Setosa|       0.0|[5.4,3.9,1.3,0.4]|
|         5.7|        2.8|         4.1|        1.3|Versicolor|       1.0|[5.7,2.

In [17]:
lr = LogisticRegression(featuresCol = 'Features', labelCol = 'varietyInd')
lrModel = lr.fit(train)

In [18]:
train_r = lrModel.transform(train)
test_r = lrModel.transform(test)

In [19]:
train_r.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.3|        3.0|         1.1|        0.1|    Setosa|       0.0|[4.3,3.0,1.1,0.1]|[67.7027919587531...|[1.0,1.6424718354...|       0.0|
|         4.4|        2.9|         1.4|        0.2|    Setosa|       0.0|[4.4,2.9,1.4,0.2]|[57.0087434717148...|[1.0,1.4036227809...|       0.0|
|         4.4|        3.0|         1.3|        0.2|    Setosa|       0.0|[4.4,3.0,1.3,0.2]|[61.7475610236625...|[1.0,2.4772179810...|       0.0|
|         4.4|        3.2|         1.3|        0.2|    Setosa|       0.0|[4.4,3.2,1.3,0.2]|[68.8060989728479...|[1.0,1.2455403381.

In [20]:
test_r.show()

+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|varietyInd|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+----------+----------+-----------------+--------------------+--------------------+----------+
|         4.6|        3.2|         1.4|        0.2|    Setosa|       0.0|[4.6,3.2,1.4,0.2]|[65.8143591752207...|[1.0,8.9932431201...|       0.0|
|         5.0|        3.0|         1.6|        0.2|    Setosa|       0.0|[5.0,3.0,1.6,0.2]|[52.7723416307807...|[1.0,9.3248066295...|       0.0|
|         5.0|        3.2|         1.2|        0.2|    Setosa|       0.0|[5.0,3.2,1.2,0.2]|[64.6690738893863...|[1.0,1.7992914773...|       0.0|
|         5.0|        3.5|         1.3|        0.3|    Setosa|       0.0|[5.0,3.5,1.3,0.3]|[71.4022940655650...|[1.0,5.6805112138.

In [21]:
ob= MulticlassClassificationEvaluator(labelCol='varietyInd')

In [22]:
print('Точность предсказания на обучающей выборке:', ob.evaluate(train_r)*100, '%')
print('Точность предсказания на тестовой выборке:', ob.evaluate(test_r)*100, '%')

Точность предсказания на обучающей выборке: 98.44961240310077 %
Точность предсказания на тестовой выборке: 100.0 %


Выводы:модель обучилась на 100%.